In [82]:
import jax
import jax.numpy as jnp
from cyipopt import minimize_ipopt
jax.config.update("jax_platform_name", "cpu")
jax.config.update("jax_enable_x64", True)

# Variáveis do sistema

In [83]:
# Número de períodos
T = 1

V = jnp.expand_dims(jnp.array([1.0, 1.02, 1.035]), -1)

theta = jnp.expand_dims(jnp.array([0.0, 0.0, 0.0]), -1)

G = jnp.array(
    [[0.0, 2.564102564, 1.156774492], [0.0, 0.0, 1.430429129], [0.0, 0.0, 0.0]]
)
B = jnp.array(
    [[0.0, -20.51282051, -4.437807597], [0.0, 0.0, -11.31339402], [0.0, 0.0, 0.0]]
)
Bsh = jnp.array([[0.0, 0.1, 0.057], [0.0, 0.0, 0.182], [0.0, 0.0, 0.0]])

a = jnp.array([100.0, 12500.0, 200.0, 0.0, 0.0, 0.0])
b = jnp.array([4000.0, 2000.0, 2000.0, 0.0, 0.0, 0.0])
c = jnp.array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
# goal = jnp.expand_dims(jnp.array([1.619514 , 2.3842845, 0.9717084]), -1)
goal = jnp.array([0.054594 , 0.0803745, 0.0327564])

Pmax = [1.0, 1.08, 1.05, 1.05, 1.8, 1.64]
Pmin = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

Pg = jnp.array([1.0, 1.08, 1.05, 1.05, 1.8, 1.64])
# Pg = jnp.repeat(Pg, T, -1)

Pc = jnp.array([[1.80], [2.65], [1.08]]) * 2
demand_multiplier = jnp.array([
    0.0337,
    0.0316,
    0.0301,
    0.0296,
    0.0296,
    0.0301,
    0.0372,
    0.0432,
    0.0477,
    0.0482,
    0.0482,
    0.0477,
    0.0477,
    0.0477,
    0.0467,
    0.0467,
    0.0497,
    0.0502,
    0.0502,
    0.0482,
    0.0457,
    0.0417,
    0.0367,
    0.0316
])
Pc = jnp.multiply(Pc, demand_multiplier[0])

P0 = jnp.array([0.0, 0.0, 0.0])
# P0 = jnp.repeat(P0, T, -1)

In [84]:
Pg.shape

(6,)

## Variáveis auxiliares

In [85]:
extra_variables = {
    "a": a,
    "b": b,
    "c": c,
    "T": T,
    "V": V,
    "theta": theta,
    "G": G,
    "B": B , 
    "Bsh": Bsh,
    "Pmax": Pmax,
    "Pmin": Pmin,
    "Pc": Pc,
    "gen_per_bus": [[], [0, 1, 2], [3, 4, 5]],
    "thermo_gen": [0, 1, 2],
    "hydro_gen": [3, 4, 5],
    'P0': P0,
    "goal": goal,
}

# Função objetivo

In [86]:
@jax.jit
def generator_cost_function(Pg: jnp.DeviceArray, extra_variables=extra_variables) -> float:
    a = extra_variables["a"]
    b = extra_variables["b"]
    c = extra_variables["c"]
    
    cost = jnp.multiply(a, Pg**2) + jnp.multiply(b, Pg) + c
    cost = jnp.sum(cost).astype(float)

    return cost

@jax.jit
def generator_cost_per_period(Pg: jnp.DeviceArray, extra_variables=extra_variables) -> float:
    a = extra_variables["a"]
    b = extra_variables["b"]
    c = extra_variables["c"]
    
    cost = jnp.multiply(a, Pg**2) + jnp.multiply(b, Pg) + c

    return cost

total_cost = generator_cost_function(Pg, extra_variables)
period_cost = generator_cost_per_period(Pg, extra_variables)
print(f"Total cost: {total_cost}")
print(f"Period cost: {period_cost}")

f = generator_cost_function
J = jax.jit(jax.grad(f, argnums=0))
H = jax.jit(jax.hessian(f, argnums=0))

Total cost: 23160.5
Period cost: [ 4100.  16740.   2320.5     0.      0.      0. ]


# Restrições

In [87]:
@jax.jit
def active_power_balance(Pg: jnp.DeviceArray, extra_variables=extra_variables) -> float:

    """

    The forward flow equation is

    P_km = Gkm V²_k - Gkm Vk Vm cos(theta_km) - Bkm Vk Vm sen(theta_km)

    The backward flow equation is

    P_mk = Gkm V²_m - Gkm Vk Vm cos(theta_km) + Bkm Vk Vm sen(theta_km)

    Where k is the starting bus and m is the ending bus connected by a transmission line,
    theta_km = theta_k - theta_m

    An expression like theta_k - theta_m can be efficiently computed as:
    theta_km = theta - theta^T, by using Jax/Numpy broadcasting capabilities
    """

    V = extra_variables["V"]
    theta = extra_variables["theta"]
    G = extra_variables["G"]
    B = extra_variables["B"]

    Vk = V
    Vm = Vk.T

    theta_k = jnp.deg2rad(theta)
    theta_m = theta_k.T
    theta_km = theta_k - theta_m

    VkVm = jnp.multiply(Vk, Vm)

    VkVm_cos_theta = jnp.multiply(VkVm, jnp.cos(theta_km))
    VkVm_sin_theta = jnp.multiply(VkVm, jnp.sin(theta_km))

    P_km = (
        jnp.multiply(G, jnp.square(Vk))
        - jnp.multiply(G, VkVm_cos_theta)
        - jnp.multiply(B, VkVm_sin_theta)
    )

    P_mk = (
        jnp.multiply(G, jnp.square(Vm))
        - jnp.multiply(G, VkVm_cos_theta)
        + jnp.multiply(B, VkVm_sin_theta)
    ).T

    P = P_km + P_mk

    P_from_bus = jnp.sum(P, axis=1)
    # P_from_bus = jnp.repeat(P_from_bus, 24, -1)

    DeltaP = 0.0 - extra_variables["Pc"][0, 0] - P_from_bus[0]
    DeltaP += Pg[0:3].sum() - extra_variables["Pc"][1, 0] - P_from_bus[1]
    DeltaP += Pg[3:].sum() - extra_variables["Pc"][2, 0] - P_from_bus[2]
    
    # return DeltaP.sum().astype(float)
    return DeltaP

d_t = active_power_balance(Pg, extra_variables)
print(d_t)

r1 = active_power_balance
J_r1 = jax.jit(jax.grad(r1, argnums=0))
H_r1 = jax.jit(jax.hessian(r1, argnums=0))

@jax.jit
def Hvp_r1(Pg, v, extra_variables=extra_variables):
    return H_r1(Pg, extra_variables) * v[0]

7.2445134636676745


In [88]:
@jax.jit
def calculate_hydro_goal_deviation(
    Pg: jnp.DeviceArray, extra_variables=extra_variables
) -> float:

    """
    Calculates the total deviation between the generated power by hydro plants and their
    generation goal throughout the 24-hour time window

    args:
        - Pg: jax array of shape (Nh, T) that contains the power generated by the
        Nh hydro generators on T periods
        - goal: jax array of shape (Nh, 1) that contains the power generation goal
        of the N generators (zero for thermal and non-zero for hydro)
    returns:
        - deviation: float which is calculated by:

        deviation = sum(Pg_t) - goal for t in 24 periods and for each generator
    """
    goal = extra_variables["goal"]

    # Sum the generations accross the 24 periods
    # Should have a shape of (N, 1)
    Pg_h = Pg[3:]

    deviation = jnp.abs(Pg_h - goal)
    deviation = jnp.sum(deviation).astype(float)

    return deviation

dv_t = calculate_hydro_goal_deviation(Pg, extra_variables)
print(dv_t)

r2 = calculate_hydro_goal_deviation
J_r2 = jax.jit(jax.grad(r2, argnums=0))
H_r2 = jax.jit(jax.hessian(r2, argnums=0))

@jax.jit
def Hvp_r2(Pg, v, extra_variables=extra_variables):
    return H_r2(Pg, extra_variables) * v[0]

4.3222751


In [89]:
cons = [
        {"type": "eq", "fun": r1, "jac": J_r1, "hess": Hvp_r1},
        {"type": "eq", "fun": r2, "jac": J_r2, "hess": Hvp_r2},
    ]

bounds = [(Pmin[i], Pmax[i]) for i in range(len(Pmax))]

In [90]:
bounds

[(0.0, 1.0), (0.0, 1.08), (0.0, 1.05), (0.0, 1.05), (0.0, 1.8), (0.0, 1.64)]

# Optimize

In [91]:
x0 = Pg.copy()
res = minimize_ipopt(
        f,
        jac=J,
        hess=H,
        x0=x0,
        bounds=bounds,
        constraints=cons,
        options={"disp": 5},
    )


This is Ipopt version 3.14.4, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:        2
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.2781380e+04 7.17e+00 5.84e+01   0.0 0.00e+00    -  0.00e+00 0.00e+00  

In [92]:
x_r = res.x
res

     fun: 243.01957621101258
    info: {'x': array([0.01320381, 0.03075628, 0.05808758, 0.08289433, 0.10882391,
       0.08172063]), 'g': array([0.        , 0.10571397]), 'obj_val': 243.01957621101258, 'mult_g': array([-8629.49745568,  8827.64150704]), 'mult_x_L': array([2410.25514887,  864.72316642,  424.12113243,  190.17227845,
          3.3048945 ,  181.63873625]), 'mult_x_U': array([21.79339644, 20.36554441, 21.08423197, 22.5167296 , 12.88710656,
       13.9831874 ]), 'status': -1, 'status_msg': b'Maximum number of iterations exceeded (can be specified by an option).'}
 message: b'Maximum number of iterations exceeded (can be specified by an option).'
    nfev: 6763
     nit: 3000
    njev: 3002
  status: -1
 success: False
       x: array([0.01320381, 0.03075628, 0.05808758, 0.08289433, 0.10882391,
       0.08172063])

In [94]:
dev = calculate_hydro_goal_deviation(x_r, extra_variables)
print(dev)
delt = active_power_balance(x_r, extra_variables)
print(delt)

0.10571396940338075
0.0
